#CM360 Segmentology
CM360 funnel analysis using Census data.


#License

Copyright 2020 Google LLC,

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

  https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.



#Disclaimer
This is not an officially supported Google product. It is a reference implementation. There is absolutely NO WARRANTY provided for using this code. The code is Apache Licensed and CAN BE fully modified, white labeled, and disassembled by your team.

This code generated (see starthinker/scripts for possible source):
  - **Command**: "python starthinker_ui/manage.py colab"
  - **Command**: "python starthinker/tools/colab.py [JSON RECIPE]"



#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CLIENT CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter CM360 Segmentology Parameters
 1. Wait for <b>BigQuery->->->Census_Join</b> to be created.
 1. Join the <a href='https://groups.google.com/d/forum/starthinker-assets' target='_blank'>StarThinker Assets Group</a> to access the following assets
 1. Copy <a href='https://datastudio.google.com/c/u/0/reporting/3673497b-f36f-4448-8fb9-3e05ea51842f/' target='_blank'>CM360 Segmentology Sample</a>. Leave the Data Source as is, you will change it in the next step.
 1. Click Edit Connection, and change to <b>BigQuery->->->Census_Join</b>.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'account': '',
  'auth_read': 'user',  # Credentials used for reading data.
  'auth_write': 'service',  # Authorization used for writing data.
  'recipe_name': '',  # Name of report, not needed if ID used.
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
  'advertisers': [],  # Comma delimited list of CM360 advertiser ids.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute CM360 Segmentology
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import Configuration
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'description': 'Create a dataset for bigquery tables.',
      'hour': [
        4
      ],
      'auth': 'user',
      'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'function': 'Pearson Significance Test',
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
      }
    }
  },
  {
    'google_api': {
      'auth': 'user',
      'api': 'dfareporting',
      'version': 'v3.4',
      'function': 'accounts.get',
      'kwargs': {
        'id': {'field': {'name': 'account','kind': 'integer','order': 5,'default': '','description': 'Campaign Manager Account ID'}},
        'fields': 'id,name'
      },
      'results': {
        'bigquery': {
          'auth': 'user',
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
          'table': 'CM360_Account'
        }
      }
    }
  },
  {
    'dcm': {
      'auth': 'user',
      'report': {
        'filters': {
          'advertiser': {
            'values': {'field': {'name': 'advertisers','kind': 'integer_list','order': 6,'default': [],'description': 'Comma delimited list of CM360 advertiser ids.'}}
          }
        },
        'account': {'field': {'name': 'account','kind': 'string','order': 5,'default': '','description': 'Campaign Manager Account ID'}},
        'body': {
          'name': {'field': {'name': 'recipe_name','kind': 'string','prefix': 'Segmentology ','description': 'The report name.','default': ''}},
          'criteria': {
            'dateRange': {
              'kind': 'dfareporting#dateRange',
              'relativeDateRange': 'LAST_24_MONTHS'
            },
            'dimensions': [
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'advertiserId'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'advertiser'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'campaignId'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'campaign'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'placementId'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'placement'
              },
              {
                'kind': 'dfareporting#sortedDimension',
                'name': 'zipCode'
              }
            ],
            'metricNames': [
              'impressions',
              'clicks',
              'totalConversions'
            ]
          },
          'type': 'STANDARD',
          'delivery': {
            'emailOwner': False
          },
          'format': 'CSV'
        }
      }
    }
  },
  {
    'dcm': {
      'auth': 'user',
      'report': {
        'account': {'field': {'name': 'account','kind': 'string','default': ''}},
        'name': {'field': {'name': 'recipe_name','kind': 'string','order': 3,'prefix': 'Segmentology ','default': '','description': 'Name of report, not needed if ID used.'}}
      },
      'out': {
        'bigquery': {
          'auth': 'user',
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
          'table': 'CM360_KPI',
          'header': True
        }
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'from': {
        'query': 'SELECT            Id AS Partner_Id,            Name AS Partner,            Advertiser_Id,            Advertiser,            Campaign_Id,            Campaign,            Zip_Postal_Code AS Zip,            SAFE_DIVIDE(Impressions, SUM(Impressions) OVER(PARTITION BY Advertiser_Id)) AS Impression,            SAFE_DIVIDE(Clicks, Impressions) AS Click,            SAFE_DIVIDE(Total_Conversions, Impressions) AS Conversion,            Impressions AS Impressions          FROM `{dataset}.CM360_KPI`          CROSS JOIN `{dataset}.CM360_Account`        ',
        'parameters': {
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be created in BigQuery.'}}
        },
        'legacy': False
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be written in BigQuery.'}},
        'view': 'CM360_KPI_Normalized'
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'normalize': {
        'census_geography': 'zip_codes',
        'census_year': '2018',
        'census_span': '5yr'
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'correlate': {
        'join': 'Zip',
        'pass': [
          'Partner_Id',
          'Partner',
          'Advertiser_Id',
          'Advertiser',
          'Campaign_Id',
          'Campaign'
        ],
        'sum': [
          'Impressions'
        ],
        'correlate': [
          'Impression',
          'Click',
          'Conversion'
        ],
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'table': 'CM360_KPI_Normalized',
        'significance': 80
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(Configuration(project=CLOUD_PROJECT, client=CLIENT_CREDENTIALS, user=USER_CREDENTIALS, verbose=True), TASKS, force=True)
